# api_experiment_get_start_skeleton(experiment_id)

The method returns the skeleton of the payload for `api_experiment_start(experiment_id)`. 

From `help experiment.service.db.ExperimentRestAPI.api_experiment_get_start_skeleton()` we can see:


In [1]:
import experiment.service.db

help(experiment.service.db.ExperimentRestAPI.api_experiment_get_start_skeleton)

Help on function api_experiment_get_start_skeleton in module experiment.service.db:

api_experiment_get_start_skeleton(self, experiment_id: 'str') -> 'Dict[str, Any]'
    Generates a skeleton payload for the api_experiment_start(experiment_id) method along with
    instructions to populate the skeleton payload.
    
    Args:
        experiment_id:
            Unique identifier of the experiment package (as stored on the consumable computing
            REST API server)
    
    Returns:
        A Dictionary with the following format: ::
    
    
            {
                "payload": {
                    the payload in dictionary form, fields that the user can fill in contain a
                    {{$magicValue}} string e.g. "{{OptionalVariableOverrideFoo}}"
                },
                "magicValues": {
                    $magicValue: {
                        "message": "A human readable message explaining what
                                 the user can do when they com

Login to your ST4SD instance - See `ST4SD Runtime API Example.ipynb` for more details.

**Note**: If you don't remember the URL to your ST4SD instance log on your OpenShift environment and get the URL of the route `st4sd-authentication` in the namespace that you deployed ST4SD in.

In [12]:
route_st4sd_runtime_service = 'your https:// URL here'

auth_url = '/'.join((route_st4sd_runtime_service.rstrip("/"), 'authorisation/token'))
print(f"Visit this URL to get your authentication token:\n{auth_url}")


Visit this URL to get your authentication token:
https://st4sd-5446-dev.ve-5446-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/authorisation/token


Fill in your token below

In [44]:
auth_token="your token goes here"

In [27]:
# Ensure that your account is authorised to use the ST4SD Runtime Service REST API
try:
    api = experiment.service.db.ExperimentRestAPI(
        route_st4sd_runtime_service, max_retries=2, secs_between_retries=1,
        cc_auth_token=auth_token
    )
except experiment.service.errors.UnauthorisedRequest as e:
    print(f"Visit {auth_url} to retrieve your authentication token. Then use it to set the value of "
          "\"auth_token\" in the above cell. Execute that cell and then execute this one.")
else:
    print(f"You've successfully authenticated to {route_st4sd_runtime_service}")

You've successfully authenticated to https://st4sd-5446-dev.ve-5446-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud


Next, let's try to get the payload skeleton for executing the [nanopore-geometry-experiment](https://registry.st4sd.res.ibm.com/registry-ui/experiment/nanopore-geometry-experiment) experiment.

If you don't have it already in your st4sd-runtime-service registry, take a moment to import it now.

In [39]:
import json

experiment_id = "nanopore-geometry-experiment"

try:
    pkg = api.api_experiment_get(experiment_id)
except experiment.service.errors.InvalidHTTPRequest as e:
    if e.response.status_code == 404:
        pkg = None

if not pkg:
    raise ValueError(f"There is no experiment {experiment_id}")


skeleton = api.api_experiment_get_start_skeleton(experiment_id)

payload_template = skeleton["payload"]
payload_instructions = skeleton["magicValues"]

Recall that the skeleton consists of 2 fields: `payload` and `magicValues`.

The `payload` field is a template which references `magicValues` that a user (you!) can fill:

In [40]:
print(json.dumps(payload_template, indent=2))

{
  "platform": "{{OptionalPlatform}}",
  "inputs": [
    {
      "filename": "cif_files.dat",
      "content": "{{RequiredInputs_cif_files.dat}}"
    }
  ],
  "s3": {
    "dataset": "{{OptionalDatasetForDownload}}",
    "accessKeyID": "{{OptionalS3ForDownload}}",
    "secretAccessKey": "{{OptionalS3ForDownload}}",
    "bucket": "{{OptionalS3ForDownload}}",
    "endpoint": "{{OptionalS3ForDownload}}",
    "region": "{{OptionalS3ForDownload}}"
  },
  "variables": {
    "numberOfNanopores": "{{OptionalVariable_numberOfNanopores}}",
    "probeRadius_A": "{{OptionalVariable_probeRadius_A}}",
    "zeo_memory": "{{OptionalVariable_zeo_memory}}"
  }
}


The `magicValues` field explains how to put together a valid payload:

In [41]:
print(json.dumps(payload_instructions, indent=2))

{
  "{{OptionalPlatform}}": {
    "message": "You **may** configure the experiment platform using one of the values ['openshift-cpu', 'openshift', 'openshift-kubeflux']",
    "choices": [
      "openshift-cpu",
      "openshift",
      "openshift-kubeflux"
    ]
  },
  "{{RequiredInputs_input_smiles.csv}}": {
    "message": "You **must** set the content of the input file input_smiles.csv either directly, via inputs[0].content or by omitting the content dictionary and configuring the payload to find the input file in a S3 bucket. For the latter consult the magicValue {{OptionalS3OrDatasetDownload}}"
  },
  "{{OptionalS3ForDownload}}": {
    "message": "You **may** ask the runtime to download files from S3 if you do not use the .content field of entries in the inputs and data array. If you pick to set {{OptionalS3ForDownload}} fields then you must not set {{OptionalDatasetForDownload}} fields. If you decide not to set {{OptionalS3ForDownload}} fields then simply remove the fields from yo

As you can see, the minimum payload is:

```json
{
   "inputs": [
    {
      "filename": "cif_files.dat",
      "content": "{{RequiredInputs_cif_files.dat}}"
    }
  ]
}
```

The instructions for populating `inputs[0].content` are:

In [45]:
print(payload_instructions["{{RequiredInputs_cif_files.dat}}"])

{'message': 'You **must** set the content of the input file cif_files.dat either directly, via inputs[0].content or by omitting the content dictionary and configuring the payload to find the input file in a S3 bucket. For the latter consult the magicValue {{OptionalS3OrDatasetDownload}}'}


If you prefer to use a Dataset (using Datashim) or a S3 bucket to provide the input file then from the above message you can see that there are instructions in the magicValue `{{OptionalS3OrDatasetDownload}}` which you can follow to fill in the `s3` field:

In [48]:
print(json.dumps(payload_template["s3"], indent=2))

{
  "dataset": "{{OptionalDatasetForDownload}}",
  "accessKeyID": "{{OptionalS3ForDownload}}",
  "secretAccessKey": "{{OptionalS3ForDownload}}",
  "bucket": "{{OptionalS3ForDownload}}",
  "endpoint": "{{OptionalS3ForDownload}}",
  "region": "{{OptionalS3ForDownload}}"
}


The instructions are:

In [51]:
print(payload_instructions["{{OptionalS3ForDownload}}"])

{'message': 'You **may** ask the runtime to download files from S3 if you do not use the .content field of entries in the inputs and data array. If you pick to set {{OptionalS3ForDownload}} fields then you must not set {{OptionalDatasetForDownload}} fields. If you decide not to set {{OptionalS3ForDownload}} fields then simply remove the fields from your payload. See also {{OptionalS3OrDatasetDownload}}.'}
